<a href="https://colab.research.google.com/github/johnsonadam187/binance_trader_app/blob/Dev/Binance_trader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>